In [11]:
%run ./UTILS/modules.ipynb

In [12]:
class ToSilver():
    #data cleaning functions
    def __init__(self, PATH):
        ## ADD BAD RECORD PATH PERMISSIVE
        self.df = spark.read.format("csv").option("Header",True).option("inferSchema",True).load(PATH)

    def nullHandling(self):
        '''
            Function responsible to handle nulls
            int: 0
            string: NA
            datetime: 1900-01-01
        '''
        
        string_col = [col[0] for col in self.df.dtypes if col[1].startswith('string')]
        int_col = [col[0] for col in self.df.dtypes if col[1].startswith('int')]
        timestamp_col = [col[0] for col in self.df.dtypes if col[1].startswith('timestamp')]
        
        return self.df.fillna('NA', subset = string_col)\
                .fillna(0, subset = int_col).fillna('1900-01-01',subset = timestamp_col)
        
    def dropColumns(self,df,columns):
        '''
            Parameters
                df: Dataframe
                columns: Column list to select
        '''
        df1 = df.select(columns).drop_duplicates()
        return df1

In [13]:
if __name__ =="__main__":
    config = read_yaml()
    for key in config['BRONZE_PATHS']:
        
        path = config['BRONZE_PATHS'][key]
        dfname = key[:-9]
        columns = config['COLUMNS'][dfname+'_COLUMNS']
        silver_path = config['SILVER_PATHS'][key]
        
        obj = ToSilver(path)
        df = obj.nullHandling()
        df = obj.dropColumns(df,columns)
        write_df_to_parquet(df,silver_path)
        print(dfname,'written to Bronze.')

CHARGES written to Bronze.
DAMAGES written to Bronze.
ENDORSE written to Bronze.
PRIMARY_PERSON written to Bronze.
RESTRICT written to Bronze.
UNITS written to Bronze.
